# Importing Libraries

Import the following four libraries by adding lines of code noted below to your `webscraping_movies.py` file.

In [2]:
import requests
import sqlite3
import pandas as pd
from bs4 import BeautifulSoup


# Initialization of known entities
---

You must declare a few entities at the beginning. For example, you know the required `URL`, the `CSV` name for saving the record, the database name, and the table name for storing the record. You also know the entities to be saved. 

Additionally, since you require only the top 50 results, you will require a loop counter initialized to 0.
 
You may initialize all these by using the following code in `webscraping_movies.py`:

In [53]:
url = 'https://web.archive.org/web/20230902185655/https://en.everybodywiki.com/100_Most_Highly-Ranked_Films'
count = 0


In [26]:
PATH_BASE = '../week-01/material/'

csv_path = PATH_BASE + 'top_50_films.csv'
table_name = PATH_BASE + 'Top_50'
db_name = PATH_BASE + 'Movies.db'


In [69]:
# EXERCISE 1

csv_path = PATH_BASE + 'top_50_films_EXERCISE_01.csv'
table_name = PATH_BASE + 'Top_50_EXERCISE_01'
db_name = PATH_BASE + 'Movies_EXERCISE_01.db'


In [55]:
df = pd.DataFrame(columns=["Film", "Year", "Rotten Tomatoes' Top 100"])


# Loading the webpage for Webscraping
---

To access the required information from the web page, you first need to load the entire web page as an `HTML` document in `python` using the `requests.get().text` function and then parse the text in the `HTML` format using `BeautifulSoup` to enable extraction of relevant information.

Add the following code to `webscraping_movies.py`:

In [56]:
html_page = requests.get(url).text
data = BeautifulSoup(html_page, 'html.parser')


# Scraping of required information

You now need to write the loop to extract the appropriate information from the web page. The rows of the table needed can be accessed using the `find_all()` function with the `BeautifulSoup` object using the statements below.


In [57]:
tables = data.find_all('tbody')
rows = tables[0].find_all('tr')


Here, the variable `tables` gets the body of all the tables in the web page and the variable `rows` gets all the rows of the first table.

You can now iterate over the rows to find the required data. Use the code shown below to extract the information.

In [45]:
top_K = 25

for row in rows:
    if count < top_K:
        col = row.find_all('td')
        if len(col)!=0:
            data_dict = {"Average Rank": col[0].contents[0],
                         "Film": col[1].contents[0],
                         "Year": col[2].contents[0]}
            df1 = pd.DataFrame(data_dict, index=[0])
            df = pd.concat([df,df1], ignore_index=True)
            count+=1
    else:
        break
    

In [58]:
# EXERCISE 1
top_K = 25

for row in rows:
    if count < top_K:
        col = row.find_all('td')
        if len(col)!=0:
            data_dict = {"Film": col[1].contents[0],
                         "Year": col[2].contents[0],
                         "Rotten Tomatoes' Top 100": col[3].contents[0]}
            df1 = pd.DataFrame(data_dict, index=[0])
            df = pd.concat([df,df1], ignore_index=True)
            count+=1
    else:
        break
    

The code functions as follows.

1. Iterate over the contents of the variable `rows`.
2. Check for the loop counter to restrict to 50 entries.
3. Extract all the `td` data objects in the row and save them to `col`.
4. Check if the length of `col` is 0, that is, if there is no data in a current row. This is important since, many timesm there are merged rows that are not apparent in the web page appearance.
5. Create a dictionary `data_dict` with the keys same as the columns of the dataframe created for recording the output earlier and corresponding values from the first three headers of data.
6. Convert the dictionary to a dataframe and concatenate it with the existing one. This way, the data keeps getting appended to the dataframe with every iteration of the loop.
7. Increment the loop counter.
8. Once the counter hits 50, stop iterating over rows and break the loop.


Print the contents of the dataframe using the following:

In [61]:
print(df.head())


                     Film  Year Rotten Tomatoes' Top 100
0           The Godfather  1972                       17
1            Citizen Kane  1941                        2
2              Casablanca  1942                        8
3  The Godfather, Part II  1974                       99
4     Singin' in the Rain  1952                       52


# Filter
---

Filter the output to print only the films released in the 2000s (year 2000 included).



In [70]:
df_filter = df[df['Year'] >= '2000']
df_filter


,Film,Year,Rotten Tomatoes' Top 100
16,Parasite,2019,6
18,Lord of the Rings: The Fellowship of the Ring,2001,unranked
22,Avengers: Endgame,2019,7


# Storing the data
---

After the dataframe has been created, you can save it to a CSV file using the following command:

In [67]:
df.to_csv(csv_path)


In [71]:
# EXERCISE 1
df_filter.to_csv(csv_path)


Remember that you defined the variable `csv_path` earlier.

To store the required data in a database, you first need to initialize a connection to the database, save the dataframe as a table, and then close the connection. This can be done using the following code:

In [ ]:
conn = sqlite3.connect(db_name)
df.to_sql(table_name, conn, if_exists='replace', index=False)
conn.close()


In [73]:
# EXERCISE 1
conn = sqlite3.connect(db_name)
df_filter.to_sql(table_name, conn, if_exists='replace', index=False)
conn.close()
